This script can be used to generate the following events given some KPI constarints for one palnned date for one particular line:

*   FTT
*   Defective
*   Defects
*   Reject
*   Rectified
*   Manpower
*   Reject reasons
*   Change

The inputs we need are:

1. A plan for one day/ multipe days  for the line of interest
2. An hourly target table of KPIs for each day (date column and hour column both)
3. General schedule (Assuming we reuse the same schedule for all days)
4. An example template for each event type (just to order the columns)





   









In [614]:
# All imports
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import string
import uuid

In [615]:
# Read constraints sheet
xls = pd.read_excel("/content/sample_data/constraints_mul_days.xlsx",
                    sheet_name=["plan","schedule", "target_kpis",
                                "ftt", "defective", "reject", "defects",
                                "rectified", "manpower", "rr"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
target_kpis = xls["target_kpis"]
ftt = xls["ftt"]
defective = xls["defective"]
reject = xls["reject"]
defects = xls["defects"]
rectified = xls["rectified"]
manpower = xls["manpower"]
reject_reason = xls["rr"]

In [616]:
# Timezone of the factory
hour_offset = 5.5

In [617]:
# Set the station per this line
station_id = "F1M8ST02"

I assume that the plan is uploaded per-day (can have multiple plans though)

In [618]:
# Function to generate random timestamps per-day that lies between a given start and end times

import random
import pandas as pd
from datetime import datetime, timedelta

def generate_random_timestamps(start_time, end_time, min_increment, max_increment, num_increments):
    """
    Generate a sorted list of timestamps with random increments for a specified date and time range.

    :param start_time: The start datetime.
    :param end_time: The end datetime.
    :param min_increment: Minimum increment in minutes.
    :param max_increment: Maximum increment in minutes.
    :param num_increments: Number of random increments to generate.
    :return: A DataFrame with the sorted timestamps.
    """
    # Generate random increments
    increments = [random.randint(min_increment, max_increment) for _ in range(num_increments)]

    # Generate timestamps
    timestamps = []
    current_time = start_time

    while current_time < end_time:
        timestamps.append(current_time)
        increment = timedelta(seconds=random.choice(increments))
        current_time += increment
        if current_time > end_time:
            break

    # Ensure timestamps are sorted
    timestamps.sort()

    # Create a DataFrame
    df = pd.DataFrame({'ts': timestamps})

    return df

In [619]:
all_dates = plan["date"].to_list()

In [620]:
# Logic to determine function parameters(with an offset for factory loacl time)

date_list =[]
start_time_list = []
end_time_list = []
num_increments_list = []

for index in range(len(all_dates)):
  date = all_dates[index].to_pydatetime().date()
  start_time_str = pd.to_timedelta(schedule["start_time"].iloc[0].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)
  end_time_str = pd.to_timedelta(schedule["end_time"].iloc[-1].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)

  start_time = datetime.combine(date, (datetime.min + start_time_str).time())
  end_time = datetime.combine(date, (datetime.min+ end_time_str).time())

  num_increments = plan["qty"].iloc[index] + random.randint(-1, 1)
  print(date)
  print(start_time)
  print(end_time)
  print(num_increments)

  date_list.append(date)
  start_time_list.append(start_time)
  end_time_list.append(end_time)
  num_increments_list.append(num_increments)



  #num_increments = 1000

min_increment = 2
max_increment = 6

2024-07-26
2024-07-26 02:00:00
2024-07-26 13:30:00
500
2024-07-27
2024-07-27 02:00:00
2024-07-27 13:30:00
601
2024-07-28
2024-07-28 02:00:00
2024-07-28 13:30:00
500
2024-07-29
2024-07-29 02:00:00
2024-07-29 13:30:00
701
2024-07-30
2024-07-30 02:00:00
2024-07-30 13:30:00
600
2024-07-31
2024-07-31 02:00:00
2024-07-31 13:30:00
499
2024-08-01
2024-08-01 02:00:00
2024-08-01 13:30:00
800


In [621]:
# Verification
print(start_time_list)


[datetime.datetime(2024, 7, 26, 2, 0), datetime.datetime(2024, 7, 27, 2, 0), datetime.datetime(2024, 7, 28, 2, 0), datetime.datetime(2024, 7, 29, 2, 0), datetime.datetime(2024, 7, 30, 2, 0), datetime.datetime(2024, 7, 31, 2, 0), datetime.datetime(2024, 8, 1, 2, 0)]


**FTT**

In [622]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)
ts_df = pd.DataFrame()

for i in range(len(date_list)):
  ts_df= pd.concat([ts_df, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df.reset_index(drop=True, inplace=True)
ts_df

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:02
2,2024-07-26 02:00:07
3,2024-07-26 02:00:09
4,2024-07-26 02:00:13
...,...
71641,2024-08-01 13:29:44
71642,2024-08-01 13:29:46
71643,2024-08-01 13:29:50
71644,2024-08-01 13:29:56


Now, I assume we have the trained synthesizers for each event type given below which we can load and use to generate more samples. But we may have to alter categorical data column values according to the plan's bctx and org. (So just learned neumerical distributions are learned.)

In [623]:
!pip install sdv

In [624]:
# To import a synthesizer
import sdv
from sdv.single_table import CTGANSynthesizer

In [625]:
# Import the pretrained synthesizer for FTT
ftt_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/ftt_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [626]:
# Generate some instances (to teh same length as timestamps)
synthetic_ftt = ftt_synthesizer.sample(num_rows=len(ts_df))

Sampling rows: 100%|██████████| 71646/71646 [00:18<00:00, 3867.85it/s]


In [627]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 12:33:28,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-06-01 12:57:38,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-06-01 06:52:09,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-06-01 06:28:48,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-06-01 08:23:40,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0


In [628]:
# Function to determine the date based on a time threshold
def get_factory_date(dt):
    dt = dt + timedelta(hours=hour_offset)
    if dt.time() < pd.to_datetime('05:00:00').time():
        return (dt - timedelta(days=1)).date()
    else:
        return dt.date()

In [629]:
# Apply factory_date logic to evt_gmt
synthetic_ftt["evt_gmt"]= ts_df["ts"]

synthetic_ftt['evt_gmt'] = pd.to_datetime(synthetic_ftt['evt_gmt'])
synthetic_ftt['factory_date'] = synthetic_ftt['evt_gmt'].apply(get_factory_date)

In [630]:
# Verification
synthetic_ftt

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-07-26 02:00:02,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-07-26 02:00:07,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-07-26 02:00:09,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-07-26 02:00:13,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71641,2024-08-01 13:29:44,2024-08-01,sdv-id-uJvOoy,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,6,0,0,0,0,0,0
71642,2024-08-01 13:29:46,2024-08-01,sdv-id-HdhMCN,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,5,0,0,0,0,0,0
71643,2024-08-01 13:29:50,2024-08-01,sdv-id-YOZhJR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,7,0,0,0,0,0,0
71644,2024-08-01 13:29:56,2024-08-01,sdv-id-sKrFQG,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,6,0,0,0,0,0,0


In [631]:
def edit_df_bctx_org(df,columns):
    # Ensure date_column is in datetime format
    df["factory_date"] = pd.to_datetime(df["factory_date"])
    plan["date"] = pd.to_datetime(plan["date"])

    # Iterate through each row in the df
    for i in range(len(df)):
        # Get the date for the current row in df
        current_date = df.loc[i, "factory_date"]

        # Find the corresponding row in the plan DataFrame
        plan_row = plan[plan["date"] == current_date]

        if not plan_row.empty:
            for column in columns:
                df.at[i, column] = plan_row.iloc[0][column]
        else:
            print(f"Date {current_date} not found in plan DataFrame")

    return df

In [632]:
# For the ease of manipulation, let us assume that we add piece by piece in
# all scenarios
# This functions sets a value column to a given value

def edit_value(df, column, value):
  df[column] = value

In [633]:
edit_value(synthetic_ftt, "ftt_count", 1)

In [634]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:00:02,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:00:07,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:00:09,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:00:13,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0


In [635]:
def find_working_hour(column, hours):
  local_time = column + timedelta(hours=hours)
  return local_time.hour * 3600 + local_time.minute * 60 + local_time.second


In [636]:
synthetic_ftt['factory_time'] = synthetic_ftt['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27000
1,2024-07-26 02:00:02,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27002
2,2024-07-26 02:00:07,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27007
3,2024-07-26 02:00:09,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27009
4,2024-07-26 02:00:13,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27013


In [637]:
# Add the seconds of the day to the schedule df
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:00').dt.time
schedule['seconds_of_day'] = schedule['start_time'].apply(lambda x: x.hour * 3600 + x.minute * 60)

In [638]:
# Verification
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [639]:
# Perform a forward-looking merge
synthetic_ftt = synthetic_ftt.sort_values('factory_time')
synthetic_ftt = pd.merge_asof(synthetic_ftt, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_ftt.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)



In [640]:
# Verification
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,1
1,2024-08-01 02:00:00,2024-08-01,sdv-id-PoLppZ,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
2,2024-07-27 02:00:00,2024-07-27,sdv-id-jayWpM,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-GNSYtP,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
4,2024-07-30 02:00:00,2024-07-30,sdv-id-Qvlyge,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1


In [641]:
synthetic_ftt = synthetic_ftt.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})


In [642]:
target_kpis

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,F1M8,1,40,8.28,0.76,2,4,1,3,12,5,17
1,2024-07-26,F1M8,2,42,8.28,0.76,1,2,1,2,12,5,17
2,2024-07-26,F1M8,3,44,8.28,0.76,1,3,1,1,12,5,17
3,2024-07-26,F1M8,4,52,8.28,0.76,1,2,1,1,12,5,17
4,2024-07-26,F1M8,5,50,8.28,0.76,1,1,1,1,11,5,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01,F1M8,7,70,8.28,1.21,2,2,2,0,14,6,20
73,2024-08-01,F1M8,8,68,8.28,1.21,2,2,1,1,14,6,20
74,2024-08-01,F1M8,9,74,8.28,1.21,1,2,1,0,14,6,20
75,2024-08-01,F1M8,10,80,8.28,1.21,1,1,1,0,14,6,20


In [643]:
# Function to extract only the necessary attribute targets
def get_relevant_target(columns):
  targets = target_kpis[columns]
  return targets

In [644]:
# Traget for FTTs
targets_ftt = get_relevant_target(["Date","Hour", "FTT"])

In [645]:
def adjust_hourly_counts(df, target_counts, count_column, hour_column='Hour', date_column='Date'):
    """
    Adjust the counts in each hour and date group of a DataFrame to match the target counts.

    :param df: DataFrame containing hour, date, and count columns.
    :param target_counts: DataFrame with hour, date, and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with adjusted counts.
    """

    # Ensure date and hour columns are of the same type (datetime64[ns])
    df[date_column] = pd.to_datetime(df[date_column])  # Convert
    df[hour_column] = pd.to_datetime(df[hour_column], format='%H').dt.hour

    # Merge the dataframes on the 'hour' and 'date' columns
    merged_df = df.merge(target_counts, on=[hour_column, date_column], how='left')

    # Create a list to store the adjusted groups
    result_groups = []

    # Iterate over each hour and date group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Calculate the difference between the current count and the target count
        current_count = len(group)
        target_count = group[count_column].iloc[0]
        difference = current_count - target_count

        # Drop rows randomly to meet the target count
        if difference > 0:
            drop_indices = group.sample(n=difference).index
            group = group.drop(drop_indices)

        # Append the group to the result list
        result_groups.append(group)

    # Concatenate the adjusted groups into a single DataFrame
    result_df = pd.concat(result_groups, ignore_index=True)

    # Drop the 'target_count' column as it's no longer needed
    #result_df = result_df.drop(columns=[count_column])

    return result_df




In [646]:
synthetic_ftt_regulated = adjust_hourly_counts(synthetic_ftt, targets_ftt,"FTT", hour_column='Hour')
synthetic_ftt_regulated = synthetic_ftt_regulated.reindex(columns=ftt.columns)
synthetic_ftt_regulated['factory_date'] = synthetic_ftt_regulated['evt_gmt'].apply(get_factory_date)


In [647]:
synthetic_ftt_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'ftt_count', 'defective_count',
       'defect_count', 'reject_count', 'rectified_count', 'input_count',
       'rw_defective_count'],
      dtype='object')

In [648]:
# # Function to add unit_id, unit columns

# def add_unit_columns(df):
#     # Get the index of the 'department_id' column
#     department_index = df.columns.get_loc('department_id')

#     # Insert 'unit_id' column with null values before 'department_id'
#     df.insert(department_index, 'unit_id', [None] * len(df))

#     # Insert 'unit' column with null values before 'department_id' (again since 'department_id' index has shifted)
#     df.insert(department_index, 'unit', [None] * len(df))

#     return df

In [649]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "department_id","department", "module_id",
                        "module", "stp", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_ftt_regulated, editable_columns)

<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BD1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'F1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'SW' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]


,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:01:32,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:03:42,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:04:54,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:05:06,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:08:13,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,2024-08-01 13:28:55,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
4182,2024-08-01 13:28:57,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
4183,2024-08-01 13:29:03,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
4184,2024-08-01 13:29:38,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0


In [650]:
# Change the line_id manually

synthetic_ftt_regulated['stp'] = synthetic_ftt_regulated['stp'] + '-' + station_id



In [651]:
synthetic_ftt_regulated['stp']

,stp
0,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
1,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
2,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
3,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
4,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
...,...
4181,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
4182,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
4183,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02
4184,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02


In [652]:
#synthetic_ftt_regulated.to_csv('synthetic_ftt_regulated.csv', index=False)

In [653]:
synthetic_ftt_regulated

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:01:32,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:03:42,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:04:54,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:05:06,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:08:13,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,2024-08-01 13:28:55,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
4182,2024-08-01 13:28:57,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
4183,2024-08-01 13:29:03,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
4184,2024-08-01 13:29:38,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0


**DEFECTIVES**

In [654]:
# Generate a large number of timestamps
ts_defective = pd.DataFrame()

for i in range(len(date_list)):
  ts_defective= pd.concat([ts_defective, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_defective.reset_index(drop=True, inplace=True)
ts_defective.head()

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:03
2,2024-07-26 02:00:06
3,2024-07-26 02:00:11
4,2024-07-26 02:00:14


In [655]:
# Import the pretrained synthesizer for defective
defective_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defective_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [656]:
# Generate some instances (to the same length as timestamps)
synthetic_defective = defective_synthesizer.sample(num_rows=len(ts_defective))

Sampling rows: 100%|██████████| 72217/72217 [00:09<00:00, 7319.80it/s]


In [657]:
# Apply factory_date logic to evt_gmt
synthetic_defective["evt_gmt"]= ts_defective["ts"]

synthetic_defective['evt_gmt'] = pd.to_datetime(synthetic_defective['evt_gmt'])
synthetic_defective['factory_date'] = synthetic_defective['evt_gmt'].apply(get_factory_date)

In [658]:
# Edit defective value

edit_value(synthetic_defective, "defective_count", 1)

In [659]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-wnVsWx,0,1,1,0,0,0,0
1,2024-07-26 02:00:03,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-GrLoVq,0,1,1,0,0,0,0
2,2024-07-26 02:00:06,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-HRikVU,0,1,1,0,0,0,0
3,2024-07-26 02:00:11,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-DgKEuo,0,1,3,0,0,0,0
4,2024-07-26 02:00:14,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,sdv-id-AQYBhy,0,1,1,0,0,0,0


In [660]:
# Function to generate gid
def generate_random_string(length=8):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

In [661]:
# Generate gid
synthetic_defective["gid"]= [generate_random_string() for _ in range(len(synthetic_defective))]

In [662]:
# Drop rows that may have generated the same random gid (rare but a precaution)
synthetic_defective= synthetic_defective.drop_duplicates(subset=['gid'])

In [663]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,QMPJCTYM,0,1,1,0,0,0,0
1,2024-07-26 02:00:03,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,T372NUWT,0,1,1,0,0,0,0
2,2024-07-26 02:00:06,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,8ECS62J2,0,1,1,0,0,0,0
3,2024-07-26 02:00:11,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,QG49FWMD,0,1,3,0,0,0,0
4,2024-07-26 02:00:14,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,4GWAQT4C,0,1,1,0,0,0,0


In [664]:
# Convert timestamp to seconds of day
synthetic_defective['factory_time'] = synthetic_defective['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,QMPJCTYM,0,1,1,0,0,0,0,27000
1,2024-07-26 02:00:03,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,defective,T372NUWT,0,1,1,0,0,0,0,27003
2,2024-07-26 02:00:06,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,defective,8ECS62J2,0,1,1,0,0,0,0,27006
3,2024-07-26 02:00:11,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,defective,QG49FWMD,0,1,3,0,0,0,0,27011
4,2024-07-26 02:00:14,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,defective,4GWAQT4C,0,1,1,0,0,0,0,27014


In [665]:
# Perform a forward-looking merge
synthetic_defective = synthetic_defective.sort_values('factory_time')
synthetic_defective = pd.merge_asof(synthetic_defective, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_defective.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_defective = synthetic_defective.rename(columns={'hour': 'Hour', 'factory_date': 'Date' })
synthetic_defective.head()



,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,QMPJCTYM,0,1,1,0,0,0,0,1
1,2024-07-29 02:00:00,2024-07-29,sdv-id-rIBtVU,1,Morning,1,mas,1,kreeda,ABC001,...,defective,NGY1B7IR,0,1,2,0,0,0,0,1
2,2024-07-31 02:00:00,2024-07-31,sdv-id-nMZCmN,1,Morning,1,mas,1,kreeda,ABC001,...,defective,33N11A3K,0,1,2,0,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-LOtaLF,1,Morning,1,mas,1,kreeda,ABC001,...,defective,BMO6QT2C,0,1,2,0,0,0,0,1
4,2024-08-01 02:00:00,2024-08-01,sdv-id-mhfZvJ,1,Morning,1,mas,1,kreeda,ABC001,...,defective,L6WP2JQT,0,1,2,0,0,0,0,1


In [666]:
# Target for defectives
targets_defective = get_relevant_target(["Date","Hour", "Defective", "Defect"])
targets_defective

,Date,Hour,Defective,Defect
0,2024-07-26,1,2,4
1,2024-07-26,2,1,2
2,2024-07-26,3,1,3
3,2024-07-26,4,1,2
4,2024-07-26,5,1,1
...,...,...,...,...
72,2024-08-01,7,2,2
73,2024-08-01,8,2,2
74,2024-08-01,9,1,2
75,2024-08-01,10,1,1


In [667]:
#pd.set_option('display.max_rows', None)

In [668]:
# Adjust the defective count to tally with the given target
defective_adjusted = adjust_hourly_counts(synthetic_defective, targets_defective,
                                          "Defective", hour_column="Hour")
defective_adjusted.head(20)
#defective_adjusted_final = defective_adjusted.drop(columns=['Hour', 'Defective',
                                                            #'Defect'])

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Defective,Defect
0,2024-07-26 02:14:27,2024-07-26,sdv-id-RtJVYM,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,2,4
1,2024-07-26 02:47:34,2024-07-26,sdv-id-kZvMuS,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,2,4
2,2024-07-26 03:17:16,2024-07-26,sdv-id-HqTzbn,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,2,1,2
3,2024-07-26 04:26:16,2024-07-26,sdv-id-sGxBFq,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,3,1,3
4,2024-07-26 05:21:04,2024-07-26,sdv-id-kdKApW,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,4,1,2
5,2024-07-26 07:27:41,2024-07-26,sdv-id-hNVYUQ,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,5,1,1
6,2024-07-26 09:23:42,2024-07-26,sdv-id-IHkbKT,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,7,1,1
7,2024-07-26 10:13:00,2024-07-26,sdv-id-mJRWgt,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,8,2,3
8,2024-07-26 10:26:56,2024-07-26,sdv-id-MfoJct,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,8,2,3
9,2024-07-26 10:57:41,2024-07-26,sdv-id-SSQceX,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,3,0,0,0,0,9,1,2


In [669]:
# Dataframe we can use to regulate defect counts
defective_adjusted.drop(columns=['Defective','Defect'], inplace=True)

In [670]:
def distribute_defects(total_defects, num_defectives):
  """
    Distribute a given number of total defects between a specified number
    of defective events.

    :param total_sum: The total defects to be distributed.
    :param num_events: The number of defective events.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_defects = total_defects // num_defectives
  remaining = total_defects % num_defectives

  distribution = [base_defects] * num_defectives

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_defectives)] += 1

  return distribution



In [671]:
def generate_hourly_counts(defectives, targets, hour_column='Hour',date_column= 'Date', count_column='defect_count'):
    """
    Generate counts for each hour of each day group of a DataFrame to match the target counts by
    distributing the target sum among all rows in the hour + date group.

    :param defectives: DataFrame containing hour and count columns.
    :param targets: DataFrame with hour and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour and date' column
    merged_df = defectives.merge(targets, on=[date_column, hour_column], how='left')

    # Create a new column for generated counts
    merged_df['generated_count'] = 0

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['Defect'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_defects(target_sum, len(group))
        merged_df.loc[group.index, 'generated_count'] = generated_counts

    # # Drop the 'target_count' column as it's no longer needed
    # result_df = merged_df.drop(columns=['target_count'])

    return merged_df


In [672]:
merged_df = generate_hourly_counts(defective_adjusted, targets_defective)
merged_df['defect_count']= merged_df['generated_count']



In [673]:
merged_df= merged_df.rename(columns={'Date': 'factory_date'})

In [674]:
# Function to add unit_id, unit columns

def add_unit_columns(df):
    # Get the index of the 'department_id' column
    department_index = df.columns.get_loc('department_id')

    # Insert 'unit_id' column with null values before 'department_id'
    df.insert(department_index, 'unit', [None] * len(df))

    # Insert 'unit' column with null values before 'department_id' (again since 'department_id' index has shifted)
    df.insert(department_index, 'unit_id', [None] * len(df))

    return df

In [675]:
merged_df = add_unit_columns(merged_df)

In [676]:
merged_df.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count', 'Hour', 'Defective', 'Defect',
       'generated_count'],
      dtype='object')

In [677]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone","department_id","department", "section_id",
                        "section", "module_id","module", "stp",  "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

synthetic_defective_regulated = edit_df_bctx_org(merged_df, editable_columns)

defects_for_defect = synthetic_defective_regulated.drop(columns=['generated_count','Defective', 'Defect'])


merged_df = synthetic_defective_regulated.drop(columns=['generated_count','Hour', 'Defective', 'Defect'])

#merged_df.to_csv('synthetic_defective_regulated.csv', index=False)

<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BD1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'F1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'SW' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]


In [678]:
# Change the line_id manually
merged_df['stp'] = merged_df['stp'] + '-' +   station_id

In [679]:
synthetic_defective_regulated = merged_df.copy()


In [680]:
synthetic_defective_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**DEFECTS**

In [681]:
# To create the defects dataframe, explode each of the  events in defective data,
# defect_count times
def generate_defects_base(columns):
  df_repeated = pd.DataFrame()
  for col in columns:
    repeated_df = []
    for index, row in merged_df.iterrows():
        repeated_df.extend([row[col]] * row['defect_count'])
    df_repeated[col] = repeated_df
  return df_repeated

In [682]:
columns_for_defects = ["evt_gmt",	"factory_date",	"shift_id",	"shift",
                        "tenant_id",	"tenant",	"cluster_id",	"cluster",
                        "factory_id",	"factory",	"building_id",	"building",
                        "unit_id", "unit","department_id",	"department",
                        "floor_id",	"floor","zone_id",	"zone",	"section_id",
                         "section",	"module_id","module","stp","size",	"hsk",
                        "buf","apt","stpv","buyer_id","buyer","vendor_id",
                        "vendor","po_id","po","so_id","so","li_id","li",
                        "fabric_type_id",	"fabric_type",	"product_category_id",
                        "product_category",	"product_type_id",	"product_type",
                        "style_id",	"style","color_id",	"color","size_id","gid"]
defects_base = generate_defects_base(columns_for_defects)

In [683]:
# Findout already processed columns
added_columns = defects_base.columns

In [684]:
# Findout what other columns should be in defects data

for column_name in added_columns:
    defects_temp = defects.drop(column_name, axis=1)
defects_temp.head(200)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-06-01 02:15:00,2024-06-01 08:12:15,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.76,0.49
1,2024-06-02 02:16:00,2024-06-01 08:33:59,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#808080,Bobbling,18,117,Att Match Bow at Waist,front,1,shirt_style1,0.73,0.59
2,2024-06-02 02:16:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#FFFFCC,Hi Low,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.07
3,2024-06-03 02:34:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,1.00,0.75
4,2024-06-04 02:35:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.00
5,2024-06-05 02:41:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800080,Wavy,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.84,0.45
6,2024-06-06 02:43:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#000080,Pleat,17,116,Att Bow at CF Waist W / mark,back,1,shirt_style1,0.74,0.00
7,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#660066,Down Stc,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.64
8,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,front,1,shirt_style1,0.87,0.45
9,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,0.77,0.40


In [685]:
# Import the pretrained synthesizer for defective
defects_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defects_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [686]:
# Generate synthetic dataset that is the same length as aggregated defective data
synthetic_defects = defects_synthesizer.sample(num_rows=len(defects_base))

Sampling rows: 100%|██████████| 205/205 [00:00<00:00, 2419.28it/s]


In [687]:
#Concatenate the aggregated defective data with generated defect-specific data
synthetic_defects = pd.concat([defects_base, synthetic_defects], axis=1)

In [688]:
# Manually change event type
synthetic_defects["evn"]= "defect"

In [689]:
# Reorder the data columns
synthetic_defects = synthetic_defects.reindex(columns=defects.columns)
synthetic_defects.head(20)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-07-26 02:14:27,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#CCCCFF,Join Stc,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.98,0.35
1,2024-07-26 02:14:27,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#808000,Steps,11,110,Att Elastic at Both Legs,back,1,shirt_style1,0.52,1.00
2,2024-07-26 02:47:34,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#993366,Oil Spot,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.77,0.59
3,2024-07-26 02:47:34,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#FF00FF,Reverse,12,111,Join 1st Side Seam,front,1,shirt_style1,0.75,0.63
4,2024-07-26 03:17:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.92,0.75
5,2024-07-26 03:17:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#0066CC,Bow Defect,12,111,Join 1st Side Seam,back,1,shirt_style1,0.86,0.40
6,2024-07-26 04:26:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,back,1,shirt_style1,0.82,0.24
7,2024-07-26 04:26:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#800080,Wavy,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.45,0.20
8,2024-07-26 04:26:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.73
9,2024-07-26 05:21:04,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,#008080,Bartack,1,100,Join Fnt Gusset Seam,front,1,shirt_style1,0.69,0.13


In [690]:
synthetic_defects.columns

Index(['evt_gmt', 'factory_date', 'shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department',
       'floor_id', 'floor', 'zone_id', 'zone', 'section_id', 'section',
       'module_id', 'module', 'station_id', 'station', 'stp', 'size', 'hsk',
       'buf', 'apt', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'sign', 'gid', 'issue_id', 'issue_code',
       'issue_color_code', 'issue', 'operation_id', 'operation_code',
       'operation', 'image', 'silhouette_id', 'silhouette', 'x', 'y'],
      dtype='object')

In [691]:
#synthetic_defects.to_csv('sythetic_defects_regulated.csv', index=False)

**RECTIFIED**

In [692]:
# We are going to derive rectified as portion of events from the defectives
# with the working hour attached.
defectives = defects_for_defect.copy()
defectives.head()
defectives.rename(columns={'factory_date': 'Date'}, inplace=True)

In [693]:
# Target for rectified
targets_rectified = get_relevant_target(["Date", "Hour", "Rectified"])
targets_rectified

,Date,Hour,Rectified
0,2024-07-26,1,1
1,2024-07-26,2,1
2,2024-07-26,3,1
3,2024-07-26,4,1
4,2024-07-26,5,1
...,...,...,...
72,2024-08-01,7,2
73,2024-08-01,8,1
74,2024-08-01,9,1
75,2024-08-01,10,1


In [694]:
def pick_rectified_events(all_defective, targets, hour_column='Hour',date_column = 'Date', datetime_column='evt_gmt', count_column='Rectified'):
    """
    Pick the first n events from the raw events DataFrame for each hour group based on the counts DataFrame.

    :param all_defective: DataFrame containing raw events with an hour and datetime column.
    :param targets: DataFrame containing the count of events per hour.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param datetime_column: Name of the column containing datetime values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with the selected events.
    """
    # Ensure the datetime column is of datetime type
    all_defective[datetime_column] = pd.to_datetime(all_defective[datetime_column])

    # Merge the raw_df and counts_df on the date + hour column
    merged_df = all_defective.merge(targets, on=[date_column, hour_column], how='left')

    # Initialize an empty DataFrame to store the selected events
    selected_events = pd.DataFrame(columns=all_defective.columns)

    # Iterate over each date+hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['defect_count'].iloc[0]

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target count for this hour
        target_count = group[count_column].iloc[0]

        # Sort the group by the datetime column
        sorted_group = group.sort_values(by=datetime_column)

        # Select the first n events
        selected_n_events = sorted_group.head(target_count)

        # Append the selected events to the result DataFrame
        selected_events = pd.concat([selected_events, selected_n_events])

    # Drop the count column from the result DataFrame
    #selected_events = selected_events.drop(columns=[count_column])

    return selected_events


In [695]:
# Select certain defectives and transform them to rectified events
rectified_events = pick_rectified_events(defectives, targets_rectified)
rectified_events.drop(columns=['Hour', 'Rectified'], inplace=True)
rectified_events['evn'] = 'rectified'
rectified_events['defective_count'] = 0
rectified_events['defect_count'] = 0
rectified_events['rectified_count'] = 1



<ipython-input-694-ee0b19bcb418>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_events = pd.concat([selected_events, selected_n_events])


In [696]:
rectified_events.head()

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:14:27,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,rectified,X23NAEVS,0,0,0,0,1,0,0
2,2024-07-26 03:17:16,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,M,rectified,H5XWFDA1,0,0,0,0,1,0,0
3,2024-07-26 04:26:16,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,rectified,9P84INHR,0,0,0,0,1,0,0
4,2024-07-26 05:21:04,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,L,rectified,O1AQVQIE,0,0,0,0,1,0,0
5,2024-07-26 07:27:41,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,M,rectified,L0QRSVZA,0,0,0,0,1,0,0


In [697]:
# Function to increment a timestamp by 10-20 seconds randomly
def increment_timestamp(timestamp):
    random_seconds = random.randint(1, 5)
    return timestamp + timedelta(seconds=random_seconds)

In [698]:
# Apply the function to the 'timestamp' column
rectified_events['evt_gmt'] = rectified_events['evt_gmt'].apply(increment_timestamp)

In [699]:
# Reorder the data columns
synthetic_rectified = rectified_events.reindex(columns=rectified.columns)


In [700]:
synthetic_rectified.rename(columns={'Date': 'factory_date'}, inplace=True)
#synthetic_rectified.to_csv('synthetic_rectified_regulated.csv', index=False)

In [701]:
# Change station_id manually
synthetic_rectified['stp'] = synthetic_rectified['stp'] + '-' + station_id


In [702]:
synthetic_rectified.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**REJECT**

In [703]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)

ts_df_reject = pd.DataFrame()

for i in range(len(date_list)):
  ts_df_reject= pd.concat([ts_df_reject, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df_reject.reset_index(drop=True, inplace=True)
ts_df_reject.head()

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:05
2,2024-07-26 02:00:11
3,2024-07-26 02:00:14
4,2024-07-26 02:00:20


In [704]:
# Import the pretrained synthesizer for Reject
reject_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/reject_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [705]:
# Generate some instances (to teh same length as timestamps)
synthetic_reject = reject_synthesizer.sample(num_rows=len(ts_df_reject))

Sampling rows: 100%|██████████| 72082/72082 [00:10<00:00, 6787.53it/s]


In [706]:
# Apply factory_date logic to evt_gmt
synthetic_reject["evt_gmt"]= ts_df_reject["ts"]

synthetic_reject['evt_gmt'] = pd.to_datetime(synthetic_reject['evt_gmt'])
synthetic_reject['factory_date'] = synthetic_reject['evt_gmt'].apply(get_factory_date)

In [707]:
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-vesfmW,reject,0,0,0,1,0,0,0
1,2024-07-26 02:00:05,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-tRzFCM,reject,0,0,0,1,0,0,0
2,2024-07-26 02:00:11,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-zMJmno,reject,0,0,0,1,0,0,0
3,2024-07-26 02:00:14,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-nATVMQ,reject,0,0,0,1,0,0,0
4,2024-07-26 02:00:20,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-ZNygiF,reject,0,0,0,1,0,0,0


In [708]:
# Generate gid
synthetic_reject["gid"]= [generate_random_string() for _ in range(len(synthetic_reject))]

In [709]:
synthetic_reject['factory_time'] = synthetic_reject['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,O142M39A,reject,0,0,0,1,0,0,0,27000
1,2024-07-26 02:00:05,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,OCBUR3U3,reject,0,0,0,1,0,0,0,27005
2,2024-07-26 02:00:11,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,HNSMXV0G,reject,0,0,0,1,0,0,0,27011
3,2024-07-26 02:00:14,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,GDUAVOQ8,reject,0,0,0,1,0,0,0,27014
4,2024-07-26 02:00:20,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,NVTNYZ2K,reject,0,0,0,1,0,0,0,27020


In [710]:
# Perform a forward-looking merge
synthetic_reject = synthetic_reject.sort_values('factory_time')
synthetic_reject = pd.merge_asof(synthetic_reject, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_reject.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_reject = synthetic_reject.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})
synthetic_reject.head()




,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,O142M39A,reject,0,0,0,1,0,0,0,1
1,2024-07-29 02:00:00,2024-07-29,sdv-id-kkVLSb,1,Morning,1,mas,1,kreeda,ABC001,...,4O6GJ2ZW,reject,0,0,0,1,0,0,0,1
2,2024-07-31 02:00:00,2024-07-31,sdv-id-AgNVXM,1,Morning,1,mas,1,kreeda,ABC001,...,7NX6SY27,reject,0,0,0,1,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-SzRQFk,1,Morning,1,mas,1,kreeda,ABC001,...,1J1LYG25,reject,0,0,0,1,0,0,0,1
4,2024-08-01 02:00:00,2024-08-01,sdv-id-qTbxzD,1,Morning,1,mas,1,kreeda,ABC001,...,RXUHKUK4,reject,0,0,0,1,0,0,0,1


In [711]:
# Target for defectives
targets_reject = get_relevant_target(["Date", "Hour", "Reject"])
targets_reject

,Date,Hour,Reject
0,2024-07-26,1,3
1,2024-07-26,2,2
2,2024-07-26,3,1
3,2024-07-26,4,1
4,2024-07-26,5,1
...,...,...,...
72,2024-08-01,7,0
73,2024-08-01,8,1
74,2024-08-01,9,0
75,2024-08-01,10,0


In [712]:
# Adjust the reject count to tally with the given target
reject_adjusted = adjust_hourly_counts(synthetic_reject, targets_reject,
                                          "Reject", hour_column="Hour")
reject_adjusted.head(20)

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Reject
0,2024-07-26 02:00:05,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,1,3
1,2024-07-26 02:01:24,2024-07-26,sdv-id-KzyMAD,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,1,3
2,2024-07-26 02:45:46,2024-07-26,sdv-id-XDADUW,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,1,3
3,2024-07-26 03:05:03,2024-07-26,sdv-id-KQOLqM,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
4,2024-07-26 03:49:50,2024-07-26,sdv-id-KBTXPQ,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
5,2024-07-26 04:19:28,2024-07-26,sdv-id-QcFWdp,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,3,1
6,2024-07-26 05:29:33,2024-07-26,sdv-id-DXjwUw,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,4,1
7,2024-07-26 06:05:14,2024-07-26,sdv-id-hrtstP,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,5,1
8,2024-07-26 08:06:57,2024-07-26,sdv-id-wGwRWe,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,6,1
9,2024-07-26 08:59:49,2024-07-26,sdv-id-fELSxM,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,7,1


In [713]:
reject_adjusted.rename(columns={'Date': 'factory_date'}, inplace=True)
synthetic_reject_regulated= reject_adjusted.reindex(columns=reject.columns)
synthetic_reject_regulated.head()



,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:05,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,OCBUR3U3,reject,0,0,0,1,0,0,0
1,2024-07-26 02:01:24,2024-07-26,sdv-id-KzyMAD,1,Morning,1,mas,1,kreeda,ABC001,...,L,KGW4Z51R,reject,0,0,0,1,0,0,0
2,2024-07-26 02:45:46,2024-07-26,sdv-id-XDADUW,1,Morning,1,mas,1,kreeda,ABC001,...,XL,HE2B1QBS,reject,0,0,0,1,0,0,0
3,2024-07-26 03:05:03,2024-07-26,sdv-id-KQOLqM,1,Morning,1,mas,1,kreeda,ABC001,...,XL,N4YLE60O,reject,0,0,0,1,0,0,0
4,2024-07-26 03:49:50,2024-07-26,sdv-id-KBTXPQ,1,Morning,1,mas,1,kreeda,ABC001,...,M,704VMLGV,reject,0,0,0,1,0,0,0


In [714]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","department_id","department","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "module_id","module", "stp", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_reject_regulated, editable_columns)

edit_value(synthetic_reject_regulated, "reject_count", 1)

<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BD1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'SW' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]
<ipython-input-631-51d921429ddb>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'F1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, column] = plan_row.iloc[0][column]


In [715]:
synthetic_reject_regulated.head(10)

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:05,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,OCBUR3U3,reject,0,0,0,1,0,0,0
1,2024-07-26 02:01:24,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,L,KGW4Z51R,reject,0,0,0,1,0,0,0
2,2024-07-26 02:45:46,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,HE2B1QBS,reject,0,0,0,1,0,0,0
3,2024-07-26 03:05:03,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,N4YLE60O,reject,0,0,0,1,0,0,0
4,2024-07-26 03:49:50,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,M,704VMLGV,reject,0,0,0,1,0,0,0
5,2024-07-26 04:19:28,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,L,QUPZL1NL,reject,0,0,0,1,0,0,0
6,2024-07-26 05:29:33,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,L,Y7A4XBOW,reject,0,0,0,1,0,0,0
7,2024-07-26 06:05:14,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,Q3ZDNOTW,reject,0,0,0,1,0,0,0
8,2024-07-26 08:06:57,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,XL,AIUXMH9H,reject,0,0,0,1,0,0,0
9,2024-07-26 08:59:49,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,L,LQASF54E,reject,0,0,0,1,0,0,0


In [716]:
#synthetic_reject_regulated.to_csv('synthetic_reject_regulated.csv', index=False)

In [717]:
synthetic_reject_regulated['stp'] = synthetic_reject_regulated['stp'] +  '-' + station_id

In [718]:
synthetic_reject_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'gid', 'evn', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**MANPOWER**

Generate categorical data columns using for example, ftt events and generate the numbers directly using schedule.

In [719]:
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [720]:
def expand_schedule_with_dates(schedule_df, date_list):
    """
    Repeat the schedule DataFrame for each date in the date_list and add the relevant date.

    :param schedule_df: The schedule DataFrame with 11 hours.
    :param date_list: List of dates to repeat the schedule for.
    :return: A DataFrame with the schedule repeated for each date, with the date added to each row.
    """
    # Create an empty DataFrame to store the results
    expanded_df = pd.DataFrame()

    # Repeat the schedule DataFrame for each date
    for date in date_list:
        temp_df = schedule_df.copy()  # Copy the schedule DataFrame
        temp_df['Date'] = date  # Add the date column
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

    return expanded_df

In [721]:
new_schedule = expand_schedule_with_dates(schedule, date_list)
new_schedule

,hour,start_time,end_time,seconds_of_day,Date
0,1,07:30:00,08:30:00,27000,2024-07-26
1,2,08:30:00,09:30:00,30600,2024-07-26
2,3,09:30:00,10:30:00,34200,2024-07-26
3,4,10:30:00,11:30:00,37800,2024-07-26
4,5,11:30:00,13:00:00,41400,2024-07-26
...,...,...,...,...,...
72,7,14:00:00,15:00:00,50400,2024-08-01
73,8,15:00:00,16:00:00,54000,2024-08-01
74,9,16:00:00,17:00:00,57600,2024-08-01
75,10,17:00:00,18:00:00,61200,2024-08-01


In [722]:
# Just using synthetic_ftt_regulated dataframe to extract the underlying org
manpower_cat = synthetic_ftt_regulated.head(len(new_schedule))
manpower_cat.drop(columns=['plan_id', 'evn', 'ftt_count', 'defective_count', 'defect_count',
                           'reject_count', 'rectified_count', 'input_count',
                           'rw_defective_count', 'buyer_id','buyer','vendor_id',
                           'vendor',	'po_id'	,'po',	'so_id',	'so',	'li_id'	,
                           'li',	'fabric_type_id',	'fabric_type',	'product_category_id',
                           'product_category',	'product_type_id',	'product_type',
                           'style_id',	'style',	'color_id',	'color'	,'size_id',	'size'], inplace=True)


In [723]:
#manpower_cat

In [724]:
# Iterate over each datetime object in the Series and format it
start_time_str = [dt.strftime('%H:%M:%S') for dt in new_schedule["start_time"]]

ts_manpower = pd.DataFrame()
ts_manpower['start_time'] = start_time_str
ts_manpower





,start_time
0,07:30:00
1,08:30:00
2,09:30:00
3,10:30:00
4,11:30:00
...,...
72,14:00:00
73,15:00:00
74,16:00:00
75,17:00:00


In [725]:
# Generate timestamps for manpower events
manpower_cat['evt_gmt'] = pd.to_datetime(new_schedule['Date'].astype(str) + ' ' +ts_manpower['start_time'] )
manpower_cat

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,section,module_id,module,station_id,station,hsk,buf,apt,stp,stpv
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
73,2024-08-01 15:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
74,2024-08-01 16:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
75,2024-08-01 17:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0


In [726]:
manpower_cat['factory_date']= manpower_cat['evt_gmt'].apply(get_factory_date)
manpower_cat

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,section,module_id,module,station_id,station,hsk,buf,apt,stp,stpv
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
73,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
74,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0
75,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,NaN,F1M8,Line 08,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0


In [727]:
# Generate Hour
manpower_cat['Hour'] = new_schedule['hour']
#manpower_cat


In [728]:
# Target for defectives (This example target does not have ironman and qc)
targets_manpower = get_relevant_target(["Date", "Hour", "Operator", "Helper", "Man power"
                                        #"Iron man", "QC"
                                        ])
targets_manpower

,Date,Hour,Operator,Helper,Man power
0,2024-07-26,1,12,5,17
1,2024-07-26,2,12,5,17
2,2024-07-26,3,12,5,17
3,2024-07-26,4,12,5,17
4,2024-07-26,5,11,5,16
...,...,...,...,...,...
72,2024-08-01,7,14,6,20
73,2024-08-01,8,14,6,20
74,2024-08-01,9,14,6,20
75,2024-08-01,10,14,6,20


In [729]:
# lookup the hourly values from given target
manpower_cat.rename(columns={'factory_date': 'Date'}, inplace=True)
manpower_cat['Date'] = pd.to_datetime(manpower_cat['Date'])
synthetic_manpower = manpower_cat.merge(targets_manpower, on=["Date", 'Hour'], how='left')
synthetic_manpower.drop(columns=['Hour'], inplace=True)
synthetic_manpower.rename(columns={'Operator': 'operator', 'Helper': 'helper', 'Man power': 'manpower'}, inplace=True)
synthetic_manpower

,evt_gmt,Date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,station_id,station,hsk,buf,apt,stp,stpv,operator,helper,manpower
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,11,5,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20
73,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20
74,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20
75,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20


In [730]:
# Add missing column and values if not present in target
if "iron man" not in synthetic_manpower.columns:
    synthetic_manpower["ironman"] = 0
if "qc" not in synthetic_manpower.columns:
    synthetic_manpower["qc"] = 0

In [731]:
synthetic_manpower.rename(columns={'Date': 'factory_date'}, inplace=True)

In [732]:
synthetic_manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,12,5,17,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,11,5,16,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20,0,0
73,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20,0,0
74,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20,0,0
75,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,14,6,20,0,0


In [733]:
# edit bctx and values as needed
editable_columns = ['shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department', 'floor_id', 'floor',
       'zone_id', 'zone', 'section_id', 'section', 'module_id', 'module',"stp"
       ]

edit_df_bctx_org(synthetic_manpower, editable_columns)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,12,5,17,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,12,5,17,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,12,5,17,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,12,5,17,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,11,5,16,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,14,6,20,0,0
73,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,14,6,20,0,0
74,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,14,6,20,0,0
75,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,14,6,20,0,0


In [734]:
manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-06-01 02:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,25,23,1,1,0


In [735]:
# Order the columns
synthetic_manpower = synthetic_manpower.reindex(columns=manpower.columns)
synthetic_manpower


,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,17,12,5,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,17,12,5,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,17,12,5,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,17,12,5,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,16,11,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,20,14,6,0,0
73,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,20,14,6,0,0
74,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,20,14,6,0,0
75,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8,1.0.0,20,14,6,0,0


In [736]:
# Manually change station_id
synthetic_manpower['stp'] = synthetic_manpower['stp'] +  '-' + station_id


In [737]:
synthetic_manpower.columns

Index(['evt_gmt', 'factory_date', 'shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department',
       'floor_id', 'floor', 'zone_id', 'zone', 'section_id', 'section',
       'module_id', 'module', 'station_id', 'station', 'hsk', 'buf', 'apt',
       'stp', 'stpv', 'manpower', 'operator', 'helper', 'ironman', 'qc'],
      dtype='object')

In [738]:
#synthetic_manpower.to_csv('synthetic_manpower_regulated.csv', index=False)

**Now we are going to generate an eid for all synthetic data**

In [739]:
def add_unique_event_ids_and_save(*dfs, filenames):
    """
    Add unique event_id column to multiple DataFrames and save each to a CSV file.

    Parameters:
    *dfs : list of pd.DataFrame
        List of DataFrames to which unique event_ids will be added.
    filenames : list of str
        List of filenames corresponding to each DataFrame for saving as CSV.

    Returns:
    list of pd.DataFrame
        List of DataFrames with unique event_ids added.
    """

    # Calculate the total number of rows across all DataFrames
    total_rows = sum(len(df) for df in dfs)

    # Generate unique event_ids
    event_ids = [str(uuid.uuid4()) for _ in range(total_rows)]

    # Initialize the start index for slicing event_ids
    start_idx = 0

    # Function to move the last column to the first position
    def move_last_column_to_first(df):
        cols = df.columns.tolist()
        new_cols = [cols[-1]] + cols[:-1]
        return df[new_cols]

    # Assign unique event_ids to each DataFrame and save to CSV
    for df, filename in zip(dfs, filenames):
        end_idx = start_idx + len(df)
        df['eid'] = event_ids[start_idx:end_idx]
        start_idx = end_idx
        df = move_last_column_to_first(df)
        df.to_csv(filename, index=False)





# Filenames for saving the CSVs
filenames = ['regulated_synthetic_ftt.csv', 'regulated_synthetic_defective.csv',
             'regulated_synthetic_defect.csv', 'regulated_synthetic_rectified.csv',
             'regulated_synthetic_reject.csv','regulated_synthetic_manpower.csv']



# Add unique event_ids to DataFrames and save to CSVs
add_unique_event_ids_and_save(synthetic_ftt_regulated,synthetic_defective_regulated,
                              synthetic_defects,synthetic_rectified,
                              synthetic_reject_regulated,synthetic_manpower
                              ,filenames=filenames)



In [740]:
# Adding plan to a seperate csv file
plan.to_csv('plan.csv', index=False)

In [741]:
# Function to bring the added eid to the very begining
def move_last_column_to_front(df):
    """
    Moves the last column of the DataFrame to the front.

    Parameters:
    df (pd.DataFrame): The DataFrame to be processed.

    Returns:
    pd.DataFrame: The DataFrame with the last column moved to the front.
    """
    # Get the last column name
    last_column = df.columns[-1]

    # Reorder columns with the last column first
    new_order = [last_column] + list(df.columns[:-1])

    # Reindex the DataFrame
    df = df[new_order]

    return df


In [742]:
# Drop gid if present and combine ftt,reject,defective and rectified as production data


def generate_production_dataset(df_list, column_name):
    """
    Checks if a particular column exists in each DataFrame in the list,
    drops that column if it exists, and then merges all DataFrames vertically.

    Parameters:
    df_list (list of pd.DataFrame): List of DataFrames to be processed.
    column_name (str): The name of the column to check and drop.

    Returns:
    pd.DataFrame: The merged DataFrame.
    """
    processed_dfs = []

    for df in df_list:
        if column_name in df.columns:
            df = df.drop(columns=[column_name])
        processed_dfs.append(df)

    # Concatenate all DataFrames vertically
    merged_df = pd.concat(processed_dfs, ignore_index=True)
    merged_df = move_last_column_to_front(merged_df)

    merged_df.to_csv('production.csv', index=False)



In [743]:
# Generate production dataset

df_list = [synthetic_ftt_regulated,synthetic_defective_regulated,
           synthetic_rectified,synthetic_reject_regulated]
column_name = 'gid'
generate_production_dataset(df_list, column_name)

<ipython-input-742-f38f3c0c3982>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(processed_dfs, ignore_index=True)


**REJECT REASON**

In [744]:
synthetic_reject_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'gid', 'evn', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count', 'eid'],
      dtype='object')

In [745]:
rejects_for_rr = synthetic_reject_regulated.copy()
rejects_for_rr.drop(columns=['evn', 'ftt_count','defective_count',
                             'defect_count', 'reject_count', 'rectified_count',
                             'input_count', 'rw_defective_count'], inplace=True)
rejects_for_rr.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,product_type_id,product_type,style_id,style,color_id,color,size_id,size,gid,eid
0,2024-07-26 02:00:05,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,XL,OCBUR3U3,8ef299ad-d5ef-432b-b41b-873f51c07fb4
1,2024-07-26 02:01:24,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,L,KGW4Z51R,5541f1d7-ee83-40c9-b9e7-f14cdbeff95a
2,2024-07-26 02:45:46,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,XL,HE2B1QBS,974f9348-6781-44da-b7c2-699b05912728
3,2024-07-26 03:05:03,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,XL,N4YLE60O,3ce61e04-3e28-41ac-a1f1-e30af89c4d2c
4,2024-07-26 03:49:50,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,M,704VMLGV,3e33a477-2f6a-40a6-9261-924921de8d45


In [746]:
# Import the pretrained synthesizer for reject reasons
rr_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/rr_synthesizer.pkl'
)

In [747]:
# Generate a set of synthetic reject reasons
synthetic_rr = rr_synthesizer.sample(num_rows=len(rejects_for_rr))


Sampling rows: 100%|██████████| 92/92 [00:00<00:00, 1833.98it/s]


In [748]:
synthetic_rr_regulated = pd.concat([rejects_for_rr, synthetic_rr], axis=1)
synthetic_rr_regulated.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,color_id,color,size_id,size,gid,eid,sign,reject_reason_id,reject_reason_code,reject_reason
0,2024-07-26 02:00:05,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,XL,OCBUR3U3,8ef299ad-d5ef-432b-b41b-873f51c07fb4,1,5,304,Fabric Defect
1,2024-07-26 02:01:24,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,L,KGW4Z51R,5541f1d7-ee83-40c9-b9e7-f14cdbeff95a,1,5,304,Fabric Defect
2,2024-07-26 02:45:46,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,XL,HE2B1QBS,974f9348-6781-44da-b7c2-699b05912728,1,2,301,Uneven
3,2024-07-26 03:05:03,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,XL,N4YLE60O,3ce61e04-3e28-41ac-a1f1-e30af89c4d2c,1,4,303,Needle Defect
4,2024-07-26 03:49:50,2024-07-26,13f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,M,704VMLGV,3e33a477-2f6a-40a6-9261-924921de8d45,1,9,308,Other


In [749]:
eid_column = synthetic_rr_regulated['eid']

In [750]:
synthetic_rr_regulated = synthetic_rr_regulated.reindex(columns=reject_reason.columns)
synthetic_rr_regulated = pd.concat([eid_column, synthetic_rr_regulated], axis=1)
synthetic_rr_regulated.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,style,color_id,color,size_id,size,sign,gid,reject_reason_id,reject_reason_code,reject_reason
0,8ef299ad-d5ef-432b-b41b-873f51c07fb4,2024-07-26 02:00:05,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,XL,1,OCBUR3U3,5,304,Fabric Defect
1,5541f1d7-ee83-40c9-b9e7-f14cdbeff95a,2024-07-26 02:01:24,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,L,1,KGW4Z51R,5,304,Fabric Defect
2,974f9348-6781-44da-b7c2-699b05912728,2024-07-26 02:45:46,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,XL,1,HE2B1QBS,2,301,Uneven
3,3ce61e04-3e28-41ac-a1f1-e30af89c4d2c,2024-07-26 03:05:03,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,XL,1,N4YLE60O,4,303,Needle Defect
4,3e33a477-2f6a-40a6-9261-924921de8d45,2024-07-26 03:49:50,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,M,1,704VMLGV,9,308,Other


In [751]:
synthetic_rr_regulated.to_csv('regulated_synthetic_rr.csv', index=False)

**CHANGE**

In [764]:
# Load synthetic manpower dataset
synthetic_manpower = pd.read_csv('regulated_synthetic_manpower.csv')

In [765]:
synthetic_manpower

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,18fc2879-d4e7-4d65-903b-6e5ed5f64244,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0
1,7e1b2cdb-45ef-4f0b-b461-a2aa14b12326,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0
2,2b8b95b7-5ade-4208-9f8c-c76e06b45688,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0
3,5cf39af2-a5a9-4040-aea7-9c27f44f9b94,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0
4,2eadbcfd-b425-438c-9a8c-2235397f6e78,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,16,11,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,9e8f02ce-1efe-4d47-b069-d4d37f6bc4ac,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,20,14,6,0,0
73,2376fa19-24dd-4dcf-87f9-cb211505a525,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,20,14,6,0,0
74,9fb6b161-9a6f-40f4-81a3-483ef90d57eb,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,20,14,6,0,0
75,032f084c-713f-463b-992a-1a068cfddf38,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,20,14,6,0,0


In [766]:
synthetic_manpower['change_type'] = 'manpower'
synthetic_manpower['scope'] = 'production'
synthetic_manpower['change'] = synthetic_manpower.apply(lambda row: {
        "ironman": str(row['ironman']),
        "helper": str(row['helper']),
        "operator": str(row['operator'])
}, axis=1)

In [755]:
synthetic_manpower.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,stp,stpv,manpower,operator,helper,ironman,qc,change_type,scope,change
0,18fc2879-d4e7-4d65-903b-6e5ed5f64244,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0,manpower,production,"{'ironman': 0, 'helper': 5, 'operator': 12}"
1,7e1b2cdb-45ef-4f0b-b461-a2aa14b12326,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0,manpower,production,"{'ironman': 0, 'helper': 5, 'operator': 12}"
2,2b8b95b7-5ade-4208-9f8c-c76e06b45688,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0,manpower,production,"{'ironman': 0, 'helper': 5, 'operator': 12}"
3,5cf39af2-a5a9-4040-aea7-9c27f44f9b94,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,17,12,5,0,0,manpower,production,"{'ironman': 0, 'helper': 5, 'operator': 12}"
4,2eadbcfd-b425-438c-9a8c-2235397f6e78,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,16,11,5,0,0,manpower,production,"{'ironman': 0, 'helper': 5, 'operator': 11}"


In [767]:
synthetic_manpower.drop(columns=['manpower', 'operator', 'helper', 'ironman',
       'qc'], inplace=True)

In [768]:
synthetic_manpower.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,station_id,station,hsk,buf,apt,stp,stpv,change_type,scope,change
0,18fc2879-d4e7-4d65-903b-6e5ed5f64244,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '12'}"
1,7e1b2cdb-45ef-4f0b-b461-a2aa14b12326,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '12'}"
2,2b8b95b7-5ade-4208-9f8c-c76e06b45688,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '12'}"
3,5cf39af2-a5a9-4040-aea7-9c27f44f9b94,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '12'}"
4,2eadbcfd-b425-438c-9a8c-2235397f6e78,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '11'}"


In [769]:
# Copy the same skeleton as manpower change
synthetic_bctx_temp = synthetic_manpower.copy()

In [770]:
# Convert factort_date to a datetime for a plan merge
synthetic_bctx_temp['factory_date'] = pd.to_datetime(synthetic_bctx_temp['factory_date'])

In [771]:
# Merge plan and change dataframes on plan date and get the bctx in the required
# nested format
synthetic_bctx= pd.merge(synthetic_bctx_temp, plan[['date','buyer','po','style',
                                                    'color']], how='left',
                         left_on='factory_date', right_on='date')
synthetic_bctx['change_type'] = 'running_order'
synthetic_bctx['scope'] = 'application'
synthetic_bctx['change'] = synthetic_bctx.apply(lambda row: {
    "buyer": row['buyer'],
    "po": row['po'],
    "style": row['style'],
    "color": row['color']
}, axis=1)
synthetic_bctx.drop(columns=['date','buyer','po','style','color'], inplace=True)
synthetic_bctx['factory_date'] = synthetic_bctx['factory_date'].dt.date
synthetic_bctx

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,station_id,station,hsk,buf,apt,stp,stpv,change_type,scope,change
0,18fc2879-d4e7-4d65-903b-6e5ed5f64244,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
1,7e1b2cdb-45ef-4f0b-b461-a2aa14b12326,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
2,2b8b95b7-5ade-4208-9f8c-c76e06b45688,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
3,5cf39af2-a5a9-4040-aea7-9c27f44f9b94,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
4,2eadbcfd-b425-438c-9a8c-2235397f6e78,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,9e8f02ce-1efe-4d47-b069-d4d37f6bc4ac,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
73,2376fa19-24dd-4dcf-87f9-cb211505a525,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1219,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
74,9fb6b161-9a6f-40f4-81a3-483ef90d57eb,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1218,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
75,032f084c-713f-463b-992a-1a068cfddf38,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,EQF1220,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M8-F1M8ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."


In [772]:
regulated_synthetic_change = pd.concat([synthetic_manpower, synthetic_bctx], axis=0)
regulated_synthetic_change.to_csv('regulated_synthetic_change.csv', index=False)

In [773]:
len(regulated_synthetic_change)

154